In [52]:
import geopandas as gpd
import pandas as pd

# Census Data Cleaning

## Age and Sex
[Data downloaded from US Census Bureau](https://data.census.gov/table?q=population+by+county+by+age&g=010XX00US$0500000&tid=ACSST5Y2021.S0101). Downloaded the ACS 5-Year Estimates Subject Tables for 2021.

in our consumption data, Children are younger than 20 years old. Adults are age 20 or older.

In [53]:
# read synced-data/county_population.csv as pandas dataframe
population = pd.read_csv('../unsynced-data/destination/census/age-sex_5-year/ACSST5Y2021.S0101-Data.csv')

In [54]:
rel_columns = ["Geography", "Geographic Area Name", "Estimate!!Total!!Total population", "Estimate!!Male!!Total population", "Estimate!!Male!!Total population!!AGE!!Under 5 years", "Estimate!!Male!!Total population!!AGE!!5 to 9 years", "Estimate!!Male!!Total population!!AGE!!10 to 14 years", "Estimate!!Male!!Total population!!AGE!!15 to 19 years", "Estimate!!Male!!Total population!!AGE!!20 to 24 years", "Estimate!!Male!!Total population!!AGE!!25 to 29 years", "Estimate!!Male!!Total population!!AGE!!30 to 34 years", "Estimate!!Male!!Total population!!AGE!!35 to 39 years", "Estimate!!Male!!Total population!!AGE!!40 to 44 years", "Estimate!!Male!!Total population!!AGE!!45 to 49 years", "Estimate!!Male!!Total population!!AGE!!50 to 54 years", "Estimate!!Male!!Total population!!AGE!!55 to 59 years", "Estimate!!Male!!Total population!!AGE!!60 to 64 years", "Estimate!!Male!!Total population!!AGE!!65 to 69 years", "Estimate!!Male!!Total population!!AGE!!70 to 74 years", "Estimate!!Male!!Total population!!AGE!!75 to 79 years", "Estimate!!Male!!Total population!!AGE!!80 to 84 years", "Estimate!!Male!!Total population!!AGE!!85 years and over", "Estimate!!Female!!Total population", "Estimate!!Female!!Total population!!AGE!!Under 5 years", "Estimate!!Female!!Total population!!AGE!!5 to 9 years", "Estimate!!Female!!Total population!!AGE!!10 to 14 years", "Estimate!!Female!!Total population!!AGE!!15 to 19 years", "Estimate!!Female!!Total population!!AGE!!20 to 24 years", "Estimate!!Female!!Total population!!AGE!!25 to 29 years", "Estimate!!Female!!Total population!!AGE!!30 to 34 years", "Estimate!!Female!!Total population!!AGE!!35 to 39 years", "Estimate!!Female!!Total population!!AGE!!40 to 44 years", "Estimate!!Female!!Total population!!AGE!!45 to 49 years", "Estimate!!Female!!Total population!!AGE!!50 to 54 years", "Estimate!!Female!!Total population!!AGE!!55 to 59 years", "Estimate!!Female!!Total population!!AGE!!60 to 64 years", "Estimate!!Female!!Total population!!AGE!!65 to 69 years", "Estimate!!Female!!Total population!!AGE!!70 to 74 years", "Estimate!!Female!!Total population!!AGE!!75 to 79 years", "Estimate!!Female!!Total population!!AGE!!80 to 84 years", "Estimate!!Female!!Total population!!AGE!!85 years and over", ]
population = population[[col for col in population.columns if col in rel_columns]]

In [55]:
population.columns = population.columns.str.replace('Estimate!!', '')
population.columns = population.columns.str.replace('AGE!!', '')
population.columns = population.columns.str.replace('Male!!Total population!!', 'male_')
population.columns = population.columns.str.replace('Female!!Total population!!', 'female_')
population.columns = population.columns.str.replace('Male!!Total population', 'male_total')
population.columns = population.columns.str.replace('Female!!Total population', 'female_total')
population.columns = population.columns.str.replace('Total!!Total population', 'total_population')
population.columns = population.columns.str.replace(' to ', '-')
population.columns = population.columns.str.replace(' years', '')
population.columns = population.columns.str.replace(' and over', '+')
population.columns = population.columns.str.replace(' ', '_')
population.columns = population.columns.str.lower()


In [56]:
grouped_pop = population[['geography', 'geographic_area_name', 'total_population', 'male_total', 'female_total',]].copy()
grouped_pop['male_kids'] = population.loc[:, 'male_under_5':'male_15-19'].sum(axis=1)
grouped_pop['male_adults'] = population.loc[:, 'male_20-24':'male_85+'].sum(axis=1)
grouped_pop['female_kids'] = population.loc[:, 'female_under_5':'female_15-19'].sum(axis=1)
grouped_pop['female_adults'] = population.loc[:, 'female_20-24':'female_85+'].sum(axis=1)
grouped_pop['kid_total'] = grouped_pop['female_kids'] + grouped_pop['male_kids']
grouped_pop['adult_total'] = grouped_pop['female_adults'] + grouped_pop['male_adults']

In [57]:
grouped_pop

,geography,geographic_area_name,total_population,male_total,female_total,male_kids,male_adults,female_kids,female_adults,kid_total,adult_total
0,0500000US01001,"Autauga County, Alabama",58239,28206,30033,7461,20745,7516,22517,14977,43262
1,0500000US01003,"Baldwin County, Alabama",227131,110781,116350,27590,83191,26383,89967,53973,173158
2,0500000US01005,"Barbour County, Alabama",25259,13361,11898,2977,10384,2853,9045,5830,19429
3,0500000US01007,"Bibb County, Alabama",22412,12300,10112,2804,9496,2359,7753,5163,17249
4,0500000US01009,"Blount County, Alabama",58884,29530,29354,7789,21741,7041,22313,14830,44054
...,...,...,...,...,...,...,...,...,...,...,...
3216,0500000US72145,"Vega Baja Municipio, Puerto Rico",54544,26057,28487,5833,20224,5523,22964,11356,43188
3217,0500000US72147,"Vieques Municipio, Puerto Rico",8317,4239,4078,835,3404,926,3152,1761,6556
3218,0500000US72149,"Villalba Municipio, Puerto Rico",22341,10796,11545,2690,8106,2495,9050,5185,17156
3219,0500000US72151,"Yabucoa Municipio, Puerto Rico",31047,15000,16047,3254,11746,3061,12986,6315,24732


In [58]:
# grouped_pop.to_csv("../unsynced-data/destination/census/cleaned-data/age-sex_5-year.csv")

## Poverty Grouping
[Data downloaded from US Census Bureau](https://data.census.gov/table?q=POVERTY+STATUS+IN+THE+PAST+12+MONTHS&g=010XX00US$0500000&tid=ACSST5Y2021.S1701). Downloaded the POVERTY STATUS IN THE PAST 12 MONTHS ACS 5-Year Estimates Subject Tables for 2021.

In our consumption data, below 185% of poverty line is low income. Above 185% is high income

In [59]:
rp_pop = pd.read_csv('../unsynced-data/destination/census/poverty-race_5-year/ACSST5Y2021.S1701-Data.csv')
rp_pop = rp_pop[[col for col in rp_pop.columns if 'Annotation' not in col]]
rp_pop = rp_pop[[col for col in rp_pop.columns if 'Margin of Error' not in col]]

/var/folders/3v/8xxt4zw96bn2l6qwb8w7_5r40000gn/T/ipykernel_22619/2338635069.py:1: DtypeWarning: Columns (230,231,232,233,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,570,571,572,573,574,575,576,577,578,579,580,581,594,595,596,597,598,599,600,601,618,619,620,621,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,730,731,732,733,734,735,736,737,738,739,740,741) have mixed types. Specify dtype option on import or set low_memory=False.
  rp_pop = pd.read_csv('../unsynced-data/destination/census/poverty-race_5-year/ACSST5Y2021.S1701-Data.csv')


In [60]:
pov_columns = ["Geography", "Geographic Area Name", "Estimate!!Total!!Population for whom poverty status is determined", "Estimate!!Total!!Population for whom poverty status is determined!!ALL INDIVIDUALS WITH INCOME BELOW THE FOLLOWING POVERTY RATIOS!!185 percent of poverty level"]

In [61]:
pov = rp_pop[[col for col in rp_pop.columns if col in pov_columns]]

In [62]:
pov = pov.rename(columns={'Geography': 'geography'})
pov = pov.merge(grouped_pop[['geography', 'total_population']], on='geography', how='left')
pov = pov.rename(columns={'Estimate!!Total!!Population for whom poverty status is determined!!ALL INDIVIDUALS WITH INCOME BELOW THE FOLLOWING POVERTY RATIOS!!185 percent of poverty level': 'low_income'})
pov = pov.rename(columns={'Estimate!!Total!!Population for whom poverty status is determined': 'total_population_determined'})
pov['high_income'] = pov['total_population_determined'] - pov['low_income']
pov['unknown_income'] = pov['total_population'] - pov['total_population_determined']
pov.columns = pov.columns.str.lower()
pov.columns = pov.columns.str.replace(' ', '_')

In [63]:
pov

,geography,geographic_area_name,total_population_determined,low_income,total_population,high_income,unknown_income
0,0500000US01001,"Autauga County, Alabama",57790,16266,58239,41524,449
1,0500000US01003,"Baldwin County, Alabama",223772,51560,227131,172212,3359
2,0500000US01005,"Barbour County, Alabama",22250,10479,25259,11771,3009
3,0500000US01007,"Bibb County, Alabama",21000,8006,22412,12994,1412
4,0500000US01009,"Blount County, Alabama",58323,17464,58884,40859,561
...,...,...,...,...,...,...,...
3216,0500000US72145,"Vega Baja Municipio, Puerto Rico",54242,37868,54544,16374,302
3217,0500000US72147,"Vieques Municipio, Puerto Rico",8317,7175,8317,1142,0
3218,0500000US72149,"Villalba Municipio, Puerto Rico",22207,17013,22341,5194,134
3219,0500000US72151,"Yabucoa Municipio, Puerto Rico",31042,23864,31047,7178,5


In [64]:
# pov.to_csv("../unsynced-data/destination/census/cleaned-data/poverty_5year.csv")

## Race Grouping
[Data downloaded from US Census Bureau](https://data.census.gov/table?q=POVERTY+STATUS+IN+THE+PAST+12+MONTHS&g=010XX00US$0500000&tid=ACSST5Y2021.S1701). Downloaded the POVERTY STATUS IN THE PAST 12 MONTHS ACS 5-Year Estimates Subject Tables for 2021.

In our consumption data racial breakdown is White, Black/African American, Hispanic/Latino, and Other.

In [65]:
race_columns = ["Geography", "Geographic Area Name", "Estimate!!Total!!Population for whom poverty status is determined", "Estimate!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!White alone, not Hispanic or Latino", "Estimate!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Black or African American alone", "Estimate!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Hispanic or Latino origin (of any race)"]
race = rp_pop[[col for col in rp_pop.columns if col in race_columns]]

In [66]:
race = race.rename(columns={'Geography': 'geography'})
race = race.merge(grouped_pop[['geography', 'total_population']], on='geography', how='left')
race = race.rename(columns={'Estimate!!Total!!Population for whom poverty status is determined': 'total_population_determined'})
race = race.rename(columns={'Estimate!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Black or African American alone': 'black-african_american'})
race = race.rename(columns={'Estimate!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Hispanic or Latino origin (of any race)': 'hispanic-latino'})
race = race.rename(columns={'Estimate!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!White alone, not Hispanic or Latino': 'white'})
race['other_race'] = race['total_population_determined']- (race['black-african_american'] + race['hispanic-latino'] + race['white'])
race['unknown_race'] = race['total_population'] - race['total_population_determined']
race.columns = race.columns.str.lower()
race.columns = race.columns.str.replace(' ', '_')

In [67]:
race

,geography,geographic_area_name,total_population_determined,black-african_american,hispanic-latino,white,total_population,other_race,unknown_race
0,0500000US01001,"Autauga County, Alabama",57790,11374,1765,42244,58239,2407,449
1,0500000US01003,"Baldwin County, Alabama",223772,19109,10066,185712,227131,8885,3359
2,0500000US01005,"Barbour County, Alabama",22250,10262,1009,10271,25259,708,3009
3,0500000US01007,"Bibb County, Alabama",21000,4232,603,16018,22412,147,1412
4,0500000US01009,"Blount County, Alabama",58323,621,5599,50190,58884,1913,561
...,...,...,...,...,...,...,...,...,...
3216,0500000US72145,"Vega Baja Municipio, Puerto Rico",54242,2409,53011,1091,54544,-2269,302
3217,0500000US72147,"Vieques Municipio, Puerto Rico",8317,629,7662,509,8317,-483,0
3218,0500000US72149,"Villalba Municipio, Puerto Rico",22207,1780,22127,50,22341,-1750,134
3219,0500000US72151,"Yabucoa Municipio, Puerto Rico",31042,16118,31004,27,31047,-16107,5


In [68]:
# race.to_csv("../unsynced-data/destination/census/cleaned-data/race_5year.csv")

## Joining Cleaned Data
Joining cleaned data into one table that will include age & sex, poverty status, and race.

In [69]:
race.columns = ['race_' + col if col not in ['geographic_area_name', 'geography'] else col for col in race.columns]
pov.columns = ['income_' + col if col not in ['geographic_area_name', 'geography'] else col for col in pov.columns]
grouped_pop.columns = ['agesex_' + col if col not in ['geographic_area_name', 'geography'] else col for col in grouped_pop.columns]

In [70]:
race_pov = pov.merge(race, on='geography')
agesex_race_pov = race_pov.merge(grouped_pop, on='geography')
columns_to_keep = ['geography', 'geographic_area_name', 'agesex_total_population', 'agesex_male_total', 'agesex_female_total', 'agesex_male_kids', 'agesex_male_adults', 'agesex_female_kids', 'agesex_female_adults', 'agesex_kid_total', 'agesex_adult_total', 'income_total_population_determined', 'income_low_income', 'income_total_population', 'income_high_income', 'income_unknown_income', 'race_total_population_determined', 'race_black-african_american', 'race_hispanic-latino', 'race_white', 'race_total_population', 'race_other_race', 'race_unknown_race']
joined_pop = agesex_race_pov[columns_to_keep]

In [71]:
# rename "agesex_total_population" to "total_population"

joined_pop = joined_pop.rename(columns={'agesex_total_population': 'total_population'})
joined_pop = joined_pop.drop(columns=['income_total_population', 'race_total_population'])

In [72]:
joined_pop

,geography,geographic_area_name,total_population,agesex_male_total,agesex_female_total,agesex_male_kids,agesex_male_adults,agesex_female_kids,agesex_female_adults,agesex_kid_total,...,income_total_population_determined,income_low_income,income_high_income,income_unknown_income,race_total_population_determined,race_black-african_american,race_hispanic-latino,race_white,race_other_race,race_unknown_race
0,0500000US01001,"Autauga County, Alabama",58239,28206,30033,7461,20745,7516,22517,14977,...,57790,16266,41524,449,57790,11374,1765,42244,2407,449
1,0500000US01003,"Baldwin County, Alabama",227131,110781,116350,27590,83191,26383,89967,53973,...,223772,51560,172212,3359,223772,19109,10066,185712,8885,3359
2,0500000US01005,"Barbour County, Alabama",25259,13361,11898,2977,10384,2853,9045,5830,...,22250,10479,11771,3009,22250,10262,1009,10271,708,3009
3,0500000US01007,"Bibb County, Alabama",22412,12300,10112,2804,9496,2359,7753,5163,...,21000,8006,12994,1412,21000,4232,603,16018,147,1412
4,0500000US01009,"Blount County, Alabama",58884,29530,29354,7789,21741,7041,22313,14830,...,58323,17464,40859,561,58323,621,5599,50190,1913,561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,0500000US72145,"Vega Baja Municipio, Puerto Rico",54544,26057,28487,5833,20224,5523,22964,11356,...,54242,37868,16374,302,54242,2409,53011,1091,-2269,302
3217,0500000US72147,"Vieques Municipio, Puerto Rico",8317,4239,4078,835,3404,926,3152,1761,...,8317,7175,1142,0,8317,629,7662,509,-483,0
3218,0500000US72149,"Villalba Municipio, Puerto Rico",22341,10796,11545,2690,8106,2495,9050,5185,...,22207,17013,5194,134,22207,1780,22127,50,-1750,134
3219,0500000US72151,"Yabucoa Municipio, Puerto Rico",31047,15000,16047,3254,11746,3061,12986,6315,...,31042,23864,7178,5,31042,16118,31004,27,-16107,5


In [73]:
# joined_pop.to_csv("../unsynced-data/destination/census/cleaned-data/joined_population.csv")

# Joining Geospatial Data

In [74]:
counties = gpd.read_file('../unsynced-data/destination/census/geo-boundaries/cb_2022_us_county_500k.shp')

In [75]:
pop_counties = joined_pop.merge(counties, left_on='geography', right_on='AFFGEOID')
pop_counties = pop_counties.drop(columns=['AFFGEOID'])
pop_counties.columns = pop_counties.columns.str.lower()
pop_counties = gpd.GeoDataFrame(pop_counties, geometry='geometry')
pop_counties.crs = {'init': 'epsg:4326'}
pop_counties_conus = pop_counties[~pop_counties['state_name'].isin(['Puerto Rico', 'Alaska', 'Hawaii'])]

/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [76]:
pop_counties

,geography,geographic_area_name,total_population,agesex_male_total,agesex_female_total,agesex_male_kids,agesex_male_adults,agesex_female_kids,agesex_female_adults,agesex_kid_total,...,countyns,geoid,name,namelsad,stusps,state_name,lsad,aland,awater,geometry
0,0500000US01001,"Autauga County, Alabama",58239,28206,30033,7461,20745,7516,22517,14977,...,00161526,01001,Autauga,Autauga County,AL,Alabama,06,1539631461,25677536,"POLYGON ((-86.92120 32.65754, -86.92035 32.658..."
1,0500000US01003,"Baldwin County, Alabama",227131,110781,116350,27590,83191,26383,89967,53973,...,00161527,01003,Baldwin,Baldwin County,AL,Alabama,06,4117724893,1132887353,"POLYGON ((-88.02858 30.22676, -88.02399 30.230..."
2,0500000US01005,"Barbour County, Alabama",25259,13361,11898,2977,10384,2853,9045,5830,...,00161528,01005,Barbour,Barbour County,AL,Alabama,06,2292160151,50523213,"POLYGON ((-85.74803 31.61918, -85.74544 31.618..."
3,0500000US01007,"Bibb County, Alabama",22412,12300,10112,2804,9496,2359,7753,5163,...,00161529,01007,Bibb,Bibb County,AL,Alabama,06,1612188713,9572302,"POLYGON ((-87.42194 33.00338, -87.31854 33.006..."
4,0500000US01009,"Blount County, Alabama",58884,29530,29354,7789,21741,7041,22313,14830,...,00161530,01009,Blount,Blount County,AL,Alabama,06,1670259093,14860281,"POLYGON ((-86.96336 33.85822, -86.95967 33.857..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3208,0500000US72145,"Vega Baja Municipio, Puerto Rico",54544,26057,28487,5833,20224,5523,22964,11356,...,01804553,72145,Vega Baja,Vega Baja Municipio,PR,Puerto Rico,13,118746310,57826361,"POLYGON ((-66.44899 18.38911, -66.44814 18.391..."
3209,0500000US72147,"Vieques Municipio, Puerto Rico",8317,4239,4078,835,3404,926,3152,1761,...,01804554,72147,Vieques,Vieques Municipio,PR,Puerto Rico,13,131541389,552192821,"POLYGON ((-65.57702 18.11972, -65.57480 18.119..."
3210,0500000US72149,"Villalba Municipio, Puerto Rico",22341,10796,11545,2690,8106,2495,9050,5185,...,01804555,72149,Villalba,Villalba Municipio,PR,Puerto Rico,13,92298567,3622640,"POLYGON ((-66.52963 18.10835, -66.52686 18.110..."
3211,0500000US72151,"Yabucoa Municipio, Puerto Rico",31047,15000,16047,3254,11746,3061,12986,6315,...,01804556,72151,Yabucoa,Yabucoa Municipio,PR,Puerto Rico,13,143005188,72592515,"POLYGON ((-66.01040 18.07201, -66.00861 18.073..."


In [77]:
pop_counties_conus

,geography,geographic_area_name,total_population,agesex_male_total,agesex_female_total,agesex_male_kids,agesex_male_adults,agesex_female_kids,agesex_female_adults,agesex_kid_total,...,countyns,geoid,name,namelsad,stusps,state_name,lsad,aland,awater,geometry
0,0500000US01001,"Autauga County, Alabama",58239,28206,30033,7461,20745,7516,22517,14977,...,00161526,01001,Autauga,Autauga County,AL,Alabama,06,1539631461,25677536,"POLYGON ((-86.92120 32.65754, -86.92035 32.658..."
1,0500000US01003,"Baldwin County, Alabama",227131,110781,116350,27590,83191,26383,89967,53973,...,00161527,01003,Baldwin,Baldwin County,AL,Alabama,06,4117724893,1132887353,"POLYGON ((-88.02858 30.22676, -88.02399 30.230..."
2,0500000US01005,"Barbour County, Alabama",25259,13361,11898,2977,10384,2853,9045,5830,...,00161528,01005,Barbour,Barbour County,AL,Alabama,06,2292160151,50523213,"POLYGON ((-85.74803 31.61918, -85.74544 31.618..."
3,0500000US01007,"Bibb County, Alabama",22412,12300,10112,2804,9496,2359,7753,5163,...,00161529,01007,Bibb,Bibb County,AL,Alabama,06,1612188713,9572302,"POLYGON ((-87.42194 33.00338, -87.31854 33.006..."
4,0500000US01009,"Blount County, Alabama",58884,29530,29354,7789,21741,7041,22313,14830,...,00161530,01009,Blount,Blount County,AL,Alabama,06,1670259093,14860281,"POLYGON ((-86.96336 33.85822, -86.95967 33.857..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3130,0500000US56037,"Sweetwater County, Wyoming",42459,21850,20609,6235,15615,5903,14706,12138,...,01609192,56037,Sweetwater,Sweetwater County,WY,Wyoming,06,27005754360,166230245,"POLYGON ((-110.05438 42.01103, -110.05436 42.0..."
3131,0500000US56039,"Teton County, Wyoming",23319,12280,11039,2551,9729,2168,8871,4719,...,01605083,56039,Teton,Teton County,WY,Wyoming,06,10351790287,570858887,"POLYGON ((-111.05361 44.66627, -110.75076 44.6..."
3132,0500000US56041,"Uinta County, Wyoming",20514,10437,10077,3254,7183,2960,7117,6214,...,01605084,56041,Uinta,Uinta County,WY,Wyoming,06,5391631764,16625817,"POLYGON ((-111.04662 41.15604, -111.04659 41.2..."
3133,0500000US56043,"Washakie County, Wyoming",7768,4004,3764,1097,2907,862,2902,1959,...,01605085,56043,Washakie,Washakie County,WY,Wyoming,06,5798152470,10415890,"POLYGON ((-108.55056 44.16846, -108.50652 44.1..."


In [78]:
# pop_counties.to_file("../unsynced-data/destination/census/cleaned-data/population_counties_all.geojson", driver='GeoJSON')
# pop_counties_conus.to_file("../unsynced-data/destination/census/cleaned-data/population_counties_conus.geojson", driver='GeoJSON')